# Analysis of correlation networks using all correlations removing the mean-corr matrix plus weighted correlations


In [ ]:
%pylab inline
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import networkx as nx
from tqdm.notebook import tqdm 

# Data import and preprocessing

In [ ]:
proper_data = pd.read_hdf('../data/proper_data.hdf');

In [ ]:
# we group the the data by condition and treatment, in order to be able to compute correlations only on the 
# appropriate values
groups = proper_data.groupby(['Condition', 'Treatment'])
regions = proper_data.columns[2:]
num_regions = len(regions)

# Computation of correlation matrices 
Here we compute the correlation matrices the difference between the various notebooks will be mostly in here, as we are exploring slightly different ways to 
compute these correlations 

In [ ]:
corrs = {}
sig_thr = 1; #here we choose the significance threshold for the correlations to be kept

from itertools import combinations
from scipy.stats import pearsonr

sample_sizes = [3,4,5,6,7,8]
for sample_size in tqdm(sample_sizes): #resampling step
    corrs[sample_size] = {}
    for i, group in groups:
        corrs[sample_size][i] = {}
        for indices in combinations(range(group.shape[0]), sample_size):
            mat = np.zeros((70,70));
            for l, r in enumerate(regions):
                for m, rr in enumerate(regions):
                    c, p = pearsonr(np.array(group[r])[list(indices)],np.array(group[rr])[list(indices)]);
                    if p<sig_thr:
                        mat[l,m] = c;
            corrs[sample_size][i][indices] = pd.DataFrame(mat, columns=regions, index=regions)

In [ ]:
import pickle as pk
pk.dump(corrs, open('../data/resampled_corrs.pck','wb'))

In [ ]:
def extract_tensor_graph(graph_tower):
    # reshaping 
    L = len(graph_tower)
    x = graph_tower[list(graph_tower.keys())[0]].shape[0]
    keys = list(graph_tower.keys())
    mat = np.zeros((L, x, x))
    for l in range(L):
        mat[l, :, :] = graph_tower[keys[l]];
    return mat;

In [ ]:
dist_maths = {}
for i, group in groups:
    dist_maths[i] = np.zeros((len(sample_sizes), len(sample_sizes)));
    for j, sz in enumerate(sample_sizes):
        ind = np.triu_indices(70,1)
        for jj, sszz in enumerate(sample_sizes):
            x, y = np.mean(extract_tensor_graph(corrs[sz][i]), 0), np.mean(extract_tensor_graph(corrs[sszz][i]), 0)
            r, p = pearsonr(x[ind], y[ind]);
            dist_maths[i][j,jj] = r;

In [ ]:
j = 1;
fig = plt.figure(figsize=(12,4))
for i, group in groups:
    plt.subplot(1,4,j)
    plt.imshow(dist_maths[i])
    plt.colorbar()
    j+=1;
plt.tight_layout();

# Dependence on density of the correlation between resamplings

In [ ]:
def density_threshold(mat, density, binarized=False):
    ind = np.triu_indices_from(mat);
    values = mat[ind]
    thr_value = np.quantile(values,1.0-density);
    thr_mat = mat.copy();
    thr_mat[mat<thr_value] = 0;
    if binarized==True:
        thr_mat[mat>=thr_value] = 1; #binarization
    return thr_mat;


rhos = np.linspace(0.001, .1, 20)
# rhos = np.logspace(-3,-0.5,10)
sz = 4;
dist_maths_rho = {}
corrs_rho = {}
for rho in rhos:
    corrs_rho[rho] = {}
    for i, group in groups:
        corrs_rho[rho][i] = {}
        for inds in corrs[sz][i]:
            corrs_rho[rho][i][inds] = density_threshold(corrs[sz][i][inds].values, rho);
              
            

sz = 4;
dist_maths_rho = {}
bin_corrs_rho = {}
for rho in rhos:
    bin_corrs_rho[rho] = {}
    for i, group in groups:
        bin_corrs_rho[rho][i] = {}
        for inds in corrs[sz][i]:
            bin_corrs_rho[rho][i][inds] = density_threshold(corrs[sz][i][inds].values, rho, binarized=True);
            

## similarity for different densities (thresholded weighted)

In [ ]:
dist_maths_rho = {}
for i, group in groups:
    dist_maths_rho[i] = np.zeros((len(rhos), len(rhos)));
    for j, rho in enumerate(rhos):
        ind = np.triu_indices(70,1)
        for jj, rrho in enumerate(rhos):
            x, y = np.mean(extract_tensor_graph(corrs_rho[rho][i]), 0), np.mean(extract_tensor_graph(corrs_rho[rrho][i]), 0)
            r, p = pearsonr(x[ind], y[ind]);
            dist_maths_rho[i][j,jj] = r;
j = 1;
fig = plt.figure(figsize=(12,4))
for i, group in groups:
    plt.subplot(1,4,j)
    plt.imshow(dist_maths_rho[i])
    plt.colorbar()
    j+=1;
plt.tight_layout();

## similarity for different densities (thresholded binarized)

In [ ]:
dist_maths_rho = {}
for i, group in groups:
    dist_maths_rho[i] = np.zeros((len(rhos), len(rhos)));
    for j, rho in enumerate(rhos):
        ind = np.triu_indices(70,1)
        for jj, rrho in enumerate(rhos):
            x, y = np.mean(extract_tensor_graph(bin_corrs_rho[rho][i]), 0), np.mean(extract_tensor_graph(bin_corrs_rho[rrho][i]), 0)
            r, p = pearsonr(x[ind], y[ind]);
            dist_maths_rho[i][j,jj] = r;
j = 1;
fig = plt.figure(figsize=(12,4))
for i, group in groups:
    plt.subplot(1,4,j)
    plt.imshow(dist_maths_rho[i])
    plt.colorbar()
    j+=1;
plt.tight_layout();

## total heterogeneity of thresholded  networks 

In [ ]:
tot_het = {}
tot_cv = {}

for rho in tqdm(rhos):
    for i in corrs_rho[rho]:
        het = []
        cv = []
        for inds in corrs_rho[rho][i]:
            tg_local = extract_tensor_graph(corrs_rho[rho][i]);
            het.append(np.std(tg_local, 0))
            cv.append(np.std(tg_local, 0) / np.mean(tg_local, 0));
    tot_het[rho] = het
    tot_cv[rho] = cv

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.errorbar(rhos, [np.nanmean(tot_het[x]) for x in rhos], [np.nanstd(tot_het[x]) for x in rhos])
plt.subplot(122)
plt.errorbar(rhos, [np.nanmean(tot_cv[x]) for x in rhos], [np.nanstd(tot_cv[x]) for x in rhos])
plt.ylim(0,10)

In [ ]:
print(rhos[np.argmax([np.nanmean(tot_cv[x]) for x in rhos])])
rhos[np.argmin([np.nanmean(tot_cv[x]) for x in rhos])]

## total heterogeneity of thresholded binarized networks 

In [ ]:
tot_het = {}
tot_cv = {}

for rho in tqdm(rhos):
    for i in bin_corrs_rho[rho]:
        het = []
        cv = []
        for inds in bin_corrs_rho[rho][i]:
            tg_local = extract_tensor_graph(bin_corrs_rho[rho][i]);
            het.append(np.std(tg_local, 0))
            cv.append(np.std(tg_local, 0) / np.mean(tg_local, 0));
    tot_het[rho] = het
    tot_cv[rho] = cv

plt.figure(figsize=(12,4))
plt.subplot(121)
plt.errorbar(rhos, [np.nanmean(tot_het[x]) for x in rhos], [np.nanstd(tot_het[x]) for x in rhos])
plt.subplot(122)
plt.errorbar(rhos, [np.nanmean(tot_cv[x]) for x in rhos], [np.nanstd(tot_cv[x]) for x in rhos])
plt.ylim(0,10)

In [ ]:
rhos[np.argmax([np.nanmean(tot_cv[x]) for x in rhos])]

# comparison with Fallani-Latora method
https://github.com/devuci/3n

In [ ]:
import networkx as nx

def objective_J(G,rho=None):
    if rho==None:
        rho = nx.density(G);
    return (nx.local_efficiency(G) + nx.global_efficiency(G))/rho;


In [ ]:
sz = 8
# rhos = np.linspace(0.01,0.2,20)
Js = {}
for i, group in tqdm(groups):
    Js[i] = []
    for rho in rhos:
        adj = list(corrs[sz][i].values())[0].values
        adj = density_threshold(adj, rho, binarized=True)
        G = nx.from_numpy_array(adj)
        Js[i].append(objective_J(G, rho));

In [ ]:
for i in Js:
    plt.plot(rhos,Js[i], label=str(i))
    print(i, rhos[np.argmax(Js[i])])
plt.legend()

In [ ]:
tg = {}
sz = 8
for i in corrs[sz]:
    tg[i] = np.mean(extract_tensor_graph(corrs[sz][i]), 0)
    nums = np.reshape(tg[i], (1, 70*70))
    print(i, np.quantile(nums,0.9))

In [ ]:
av_J, std_J = [], []
for i, rho in enumerate(rhos):
    av_J.append(np.nanmean([Js[x][i] for x in Js]))
    std_J.append(np.std([Js[x][i] for x in Js]))


plt.errorbar(rhos,av_J, std_J)



## Joint comparison

In [ ]:
skip = 2

mean_tot_cv = np.array([np.nanmean(tot_cv[x]) for x in rhos])
plt.plot(rhos[skip:], mean_tot_cv[skip:]/np.max(mean_tot_cv[skip:]), 'o--')
for i in Js:
    plt.plot(rhos[skip:],Js[i][skip:]/np.max(Js[i][skip:]))


In [ ]:
plt.figure(figsize=(16,4))

plt.subplot(131)
plt.errorbar(rhos, [np.nanmean(tot_het[x]) for x in rhos], [np.nanstd(tot_het[x]) for x in rhos])
plt.plot(rhos, [np.nanmean(tot_het[x]) for x in rhos], 'bo', alpha=.4)
plt.ylabel(r'$\zeta(\Omega^\rho)$', fontsize=20)
plt.xlabel(r'$\rho$', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.subplot(132)
plt.errorbar(rhos, [np.nanmean(tot_cv[x]) for x in rhos], [np.nanstd(tot_cv[x]) for x in rhos])
plt.plot(rhos, [np.nanmean(tot_cv[x]) for x in rhos], 'bo', alpha=.4)
plt.ylabel(r'$\xi(\Omega^\rho)$', fontsize=20)
plt.ylim(0,10)
plt.xlabel(r'$\rho$', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.subplot(133)
plt.errorbar(rhos,av_J, std_J)
plt.plot(rhos, av_J, 'bo', alpha=.4)
plt.ylabel(r'$J(\rho)$', afontsize=20)
plt.xlabel(r'$\rho$', fontsize=20)
plt.xticks(fontsize=15)
plt.yticks(fontsize=15)

plt.tight_layout()